In [2]:
import pandas as pd

# all_sheets_data = pd.read_csv('./flower.xlsx')
# all_sheets_data.head()
file_path = './flower.xlsx'  # 更改为您的文件路径
all_sheets_data = pd.read_excel(file_path, sheet_name=None)

In [3]:

all_products_summary = pd.DataFrame()
# 遍历每个工作表
for sheet_name, sheet_data in all_sheets_data.items():
    # 确保 sheet_data 是 DataFrame
    if isinstance(sheet_data, pd.DataFrame) and '订单号' in sheet_data.columns and '订单状态' in sheet_data.columns:
        # 使用前向填充（ffill）方法填充订单号和订单状态
        sheet_data['订单号'].ffill(inplace=True)
        sheet_data['订单状态'].ffill(inplace=True)

        # 打印 ffill 后的表格以供检查
        # print(f"Sheet: {sheet_name} after ffill")
        # print(sheet_data.head(30))  # 打印前几行以检查
        # 只考虑已支付的订单
        paid_orders = sheet_data[sheet_data['订单状态'] == '已支付']

        # 检查并处理数量或商品金额为零的情况
        paid_orders = paid_orders[(paid_orders['数量'] > 0) & (paid_orders['商品金额'] > 0)]

        # 计算单价
        paid_orders['单价'] = paid_orders['商品金额'] / paid_orders['数量']

        # 按商品名称分组，并计算每个商品的总数量和平均单价
        grouped = paid_orders.groupby('商品名称').agg({'数量': 'sum', '单价': 'mean'}).reset_index()

        # 将分组结果添加到汇总 DataFrame
        all_products_summary = pd.concat([all_products_summary, grouped], ignore_index=True)

# 对所有商品名称进行分组并求和数量，同时计算平均商品金额
final_summary = all_products_summary.groupby('商品名称').agg({'数量': 'sum'}).reset_index()
# 按数量降序排序
final_summary_sorted = final_summary.sort_values(by='数量', ascending=False).reset_index(drop=True)

final_summary_sorted
# 计算总价格
final_summary['总价格'] = (final_summary['数量'] * final_summary['单价']).round(2)
# 输出结果到 CSV 文件
csv_file_path = 'final_summary.csv'  # 更改为您想要的文件名和路径
final_summary.to_csv(csv_file_path, index=False)
# 打印最终汇总
print(final_summary_sorted)

# 计算所有商品的总价格
total_price = final_summary['总价格'].sum()
print(f"所有商品的总价格: {total_price}")

KeyError: '单价'